In [ ]:
from matplotlib import pyplot as plt
import h5py
import re

noalign = "../taiyakiOutputs/output_justfromfasta.hdf5"
yealign = "../taiyakiOutputs/output_map_whole_genome.hdf5"

In [ ]:
labelBaseMap = {
    0: "A",
    1: "C",
    2: "G",
    3: "T"
}

def labelToBase(label):
    return labelBaseMap[label]
    
def getReadsDict(filename):
    file = h5py.File(filename, "r")
    file = file['Reads']
    reads = []
    for r in file.keys():
        elem = {}
        elem['UUID'] = r
        for k in file[r].keys():
            elem[k]=file[r][k][()]
        reads.append(elem)
    return reads

def checkedZippedIsSynced(zipped):
    dirty = False
    for z in zipped:
        if z[0]['UUID'] != z[1]['UUID']:
            dirty = True
            print(f"Fail at {z}")
    return dirty

In [ ]:
readsnoalign = getReadsDict(noalign)
readsyealign = getReadsDict(yealign)

# create intersection
keysInAligned = list(map(lambda read: read['UUID'], readsyealign))
readsnoalign = list(filter(lambda read: read['UUID'] in keysInAligned, readsnoalign))

zipped = list(zip(readsyealign, readsnoalign))

if checkedZippedIsSynced(zipped):
    print("The two lists are synced")

In [ ]:
for z in zipped:
    pref = "".join(list(map(labelToBase, list(z[1]['Reference'][:len(list(filter(lambda x: x<=z[0]['Ref_to_signal'][0], z[1]['Ref_to_signal'])))]))))
    print(f"{z[0]['UUID']} - {pref}")